In [29]:
import os
import json
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime

In [1]:
path = "github_api/"
my_filenames = [
    os.path.join(root, name)
    for root, dirs, files in os.walk(path)
    for name in files
    if name.endswith((".json"))
]
my_filenames

['github_api/111online_repos.json',
 'github_api/nhsconnect_repos.json',
 'github_api/nhsdigital_repos.json',
 'github_api/nhsengland_repos.json',
 'github_api/nhsuk_repos.json',
 'github_api/nhsx_repos.json']

In [13]:
# last modified datex
time_stamp = os.path.getmtime(my_filenames[0])
mod_date = datetime.fromtimestamp(time_stamp).strftime("%Y-%m-%d %H:%M:%S")
print(mod_date)

2021-03-04 10:24:50


In [ ]:
with open('github_api/nhsx_repos.json') as f:
  data = json.load(f)

In [44]:
from pandas.io.json import json_normalize
flat_data = json_normalize(data)
flat_data
flat_data.assign(date = mod_date)
flat_data.head()

,id,node_id,name,full_name,private,html_url,description,fork,url,forks_url,...,owner.site_admin,license.key,license.name,license.spdx_id,license.url,license.node_id,permissions.admin,permissions.push,permissions.pull,license
0,177147975,MDEwOlJlcG9zaXRvcnkxNzcxNDc5NzU=,nhsx-prototype,nhsx/nhsx-prototype,False,https://github.com/nhsx/nhsx-prototype,Design prototypes for the NHSX website,False,https://api.github.com/repos/nhsx/nhsx-prototype,https://api.github.com/repos/nhsx/nhsx-prototy...,...,False,mit,MIT License,MIT,https://api.github.com/licenses/mit,MDc6TGljZW5zZTEz,False,False,True,NaN
1,178217860,MDEwOlJlcG9zaXRvcnkxNzgyMTc4NjA=,old-nhsx-website,nhsx/old-nhsx-website,False,https://github.com/nhsx/old-nhsx-website,:warning: This repository has been deprecated ...,False,https://api.github.com/repos/nhsx/old-nhsx-web...,https://api.github.com/repos/nhsx/old-nhsx-web...,...,False,mit,MIT License,MIT,https://api.github.com/licenses/mit,MDc6TGljZW5zZTEz,False,False,True,NaN
2,189214616,MDEwOlJlcG9zaXRvcnkxODkyMTQ2MTY=,open-source-fhir-server,nhsx/open-source-fhir-server,False,https://github.com/nhsx/open-source-fhir-server,Open source fhir server,False,https://api.github.com/repos/nhsx/open-source-...,https://api.github.com/repos/nhsx/open-source-...,...,False,mit,MIT License,MIT,https://api.github.com/licenses/mit,MDc6TGljZW5zZTEz,False,False,True,NaN
3,193096724,MDEwOlJlcG9zaXRvcnkxOTMwOTY3MjQ=,nhsalexahack,nhsx/nhsalexahack,False,https://github.com/nhsx/nhsalexahack,None,True,https://api.github.com/repos/nhsx/nhsalexahack,https://api.github.com/repos/nhsx/nhsalexahack...,...,False,NaN,NaN,NaN,NaN,NaN,False,False,True,NaN
4,193874492,MDEwOlJlcG9zaXRvcnkxOTM4NzQ0OTI=,algorithm-library,nhsx/algorithm-library,False,https://github.com/nhsx/algorithm-library,A curated list of algorithms and datasets. All...,False,https://api.github.com/repos/nhsx/algorithm-li...,https://api.github.com/repos/nhsx/algorithm-li...,...,False,mit,MIT License,MIT,https://api.github.com/licenses/mit,MDc6TGljZW5zZTEz,False,False,True,NaN


In [43]:
df = pd.DataFrame()
for filename in my_filenames:
    with open(filename) as f:
        data = json.load(f)
    flat_data = json_normalize(data)
    extract_date = datetime.fromtimestamp(os.path.getmtime(filename)).strftime("%Y-%m-%d %H:%M:%S")
    flat_data = flat_data.assign(file_name = filename)
    flat_data = flat_data.assign(file_date = extract_date)
    df = df.append(flat_data)
df.head()

,id,node_id,name,full_name,private,html_url,description,fork,url,forks_url,...,permissions.admin,permissions.push,permissions.pull,license.key,license.name,license.spdx_id,license.url,license.node_id,file_name,file_date
0,138751264,MDEwOlJlcG9zaXRvcnkxMzg3NTEyNjQ=,nhs111-implementation-site,111Online/nhs111-implementation-site,False,https://github.com/111Online/nhs111-implementa...,Implementation site for 111 online,False,https://api.github.com/repos/111Online/nhs111-...,https://api.github.com/repos/111Online/nhs111-...,...,False,False,True,NaN,NaN,NaN,NaN,NaN,github_api/111online_repos.json,2021-03-04 10:24:50
1,138753441,MDEwOlJlcG9zaXRvcnkxMzg3NTM0NDE=,web-stack,111Online/web-stack,False,https://github.com/111Online/web-stack,NHS 111 Online Product,False,https://api.github.com/repos/111Online/web-stack,https://api.github.com/repos/111Online/web-sta...,...,False,False,True,apache-2.0,Apache License 2.0,Apache-2.0,https://api.github.com/licenses/apache-2.0,MDc6TGljZW5zZTI=,github_api/111online_repos.json,2021-03-04 10:24:50
2,206048937,MDEwOlJlcG9zaXRvcnkyMDYwNDg5Mzc=,nhs111-domain-dos-api,111Online/nhs111-domain-dos-api,False,https://github.com/111Online/nhs111-domain-dos...,None,False,https://api.github.com/repos/111Online/nhs111-...,https://api.github.com/repos/111Online/nhs111-...,...,False,False,True,apache-2.0,Apache License 2.0,Apache-2.0,https://api.github.com/licenses/apache-2.0,MDc6TGljZW5zZTI=,github_api/111online_repos.json,2021-03-04 10:24:50
3,206102072,MDEwOlJlcG9zaXRvcnkyMDYxMDIwNzI=,nhs111-itk-dispatcher-api,111Online/nhs111-itk-dispatcher-api,False,https://github.com/111Online/nhs111-itk-dispat...,None,False,https://api.github.com/repos/111Online/nhs111-...,https://api.github.com/repos/111Online/nhs111-...,...,False,False,True,apache-2.0,Apache License 2.0,Apache-2.0,https://api.github.com/licenses/apache-2.0,MDc6TGljZW5zZTI=,github_api/111online_repos.json,2021-03-04 10:24:50
4,209029513,MDEwOlJlcG9zaXRvcnkyMDkwMjk1MTM=,nhs111-feedback-api,111Online/nhs111-feedback-api,False,https://github.com/111Online/nhs111-feedback-api,None,False,https://api.github.com/repos/111Online/nhs111-...,https://api.github.com/repos/111Online/nhs111-...,...,False,False,True,apache-2.0,Apache License 2.0,Apache-2.0,https://api.github.com/licenses/apache-2.0,MDc6TGljZW5zZTI=,github_api/111online_repos.json,2021-03-04 10:24:50


In [47]:
file_date = datetime.fromtimestamp(os.path.getmtime(filename)).strftime("%Y-%m-%d")
df.to_csv(r'data/github-api-output_'+file_date+'.csv')